In [1]:
import os, time, sys
import pandas as pd
from digital_land.collection import Collection

from functions import run_endpoint_workflow
from sqlite_query_functions import DatasetSqlite
from convert_functions import convert_resource




/Users/jimmytidey/Projects/jupyter-analysis/venv/lib/python3.10/site-packages/pyproj/transformer.py:207: UserWarning: Best transformation is not available due to missing Grid(short_name=uk_os_OSTN15_NTv2_OSGBtoETRS.tif, full_name=, package_name=, url=https://cdn.proj.org/uk_os_OSTN15_NTv2_OSGBtoETRS.tif, direct_download=True, open_license=True, available=False)
  super().__init__(


In [3]:
collection_name = 'listed-building-collection'
dataset = 'listed-building-outline'
organisation = 'local-authority-eng:CAM'
endpoint_url = 'https://raw.githubusercontent.com/jimmytidey/jupyter-analysis/feature-endpoint-checker-api/endpoint_checker/examples/conservation_area_bad_field_names.csv'
plugin = None
additional_column_mappings=None
additional_concats=None
data_dir = 'data/endpoint_checker_' + str(int(time.time()*10000)) 


run_endpoint_workflow(
    collection_name,
    dataset,
    organisation,
    endpoint_url,
    plugin,
    data_dir,
    additional_col_mappings=additional_column_mappings,
    additional_concats=additional_concats
)

collection = Collection(os.path.join(data_dir,'collection'))
collection.load(directory=os.path.join(data_dir,'collection'))
logs = collection.log.entries
logs = pd.DataFrame.from_records(logs)
http_status = logs['status'].values[0]
print(f'http status {http_status}')

dataset_db = DatasetSqlite(os.path.join(data_dir,'dataset',f'{dataset}.sqlite3'))
results = dataset_db.get_issues()
issues = results.loc[(results.issue_type != 'default-value')]
display(issues)


HTTP Error 404: Not Found
HTTP Error 404: Not Found
data/endpoint_checker_16967120869144/collection
[{'resource': '88719225f56c0e8f01f587dc85ebb65983c334442be2ddc9571cff60bfd5fda9', 'bytes': '42967', 'endpoints': 'a10b3732afc3a9ef86a263d6bed346529271f94589a4d84ecb656c055cb06560', 'organisations': 'local-authority-eng:CAM', 'datasets': 'listed-building-outline', 'start-date': '2023-10-07', 'end-date': ''}]


ERROR:root:entity 42117820: missing reference


http status 200


,end_date,entry_date,entry_number,field,issue_type,line_number,dataset,resource,start_date,value
